In [1]:
import pandas as pd
from sklearn import linear_model
from src import DIR_DEBUG
import numpy as np
from src.sample_data import SampleData

binary_sample = SampleData("binary_classification")

df = binary_sample.train
df_test = binary_sample.test
# sample = pd.read_csv("../input/sample_submission.csv")

train_len = len(df)

df_test["target"] = -1
full_data = pd.concat([df, df_test])

cols = [c for c in df.columns if c not in ["id", "target"]]


In [2]:
full_data = full_data[1:5]
full_data = full_data.iloc[:,:5].reset_index(drop=True)
full_data

,id,bin_0,bin_1,bin_2,bin_3
0,1,0,1,0,T
1,2,0,0,0,F
2,3,0,1,0,F
3,4,0,0,0,F


In [3]:
from src.categorical import CategoricalFeatures
cat_feats = CategoricalFeatures(full_data, 
                                categorical_features=full_data.columns, 
                                encoding_type="ohe2",
                                handle_na=True)
full_data_transformed = cat_feats.fit_transform()
full_data_transformed





,id_1,id_2,id_3,id_4,bin_0_0,bin_1_0,bin_1_1,bin_2_0,bin_3_F,bin_3_T
0,1,0,0,0,1,0,1,1,0,1
1,0,1,0,0,1,1,0,1,1,0
2,0,0,1,0,1,0,1,1,1,0
3,0,0,0,1,1,1,0,1,1,0


In [ ]:
print(full_data_transformed.columns)

In [ ]:
X = full_data_transformed[:train_len, :]
X_test = full_data_transformed[train_len:, :]

clf = linear_model.LogisticRegression()
clf.fit(X, df.target.values)
preds = clf.predict_proba(X_test)[:, 1]

print(full_data_transformed.head())
full_data_transformed['predicted_probabilities'] = -1
full_data_transformed[train_len:,'predicted_probabilities'] = np.round(preds,2)

filepath = DIR_DEBUG.joinpath("categorical_results.csv")

full_data_transformed.to_csv(filepath)